In [2]:
import os
import sys
import re
from datetime import datetime, timedelta
from dateutil.rrule import rrule, WEEKLY, FR, SU
sys.path.append('/Users/laurenthericourt/projets/trading/trading')

import psycopg2
import pandas as pd
from pandas.io.sql import read_sql
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from config.load import load_conf
from db.utils import get_uri_db
from data.candle import SYMBOLS
from indicator.trend import BollingerBands
from indicator.oscillator import Macd

In [3]:
load_conf('../config/configuration.yaml')

In [4]:
dsn = get_uri_db()
schema = 'trading'

# Get data
## Candles

In [10]:
start_date = '2020-11-01'
end_date = '2020-12-01'

In [11]:
candles = pd.DataFrame()
with psycopg2.connect(dsn) as conn:
    for table in ['candle', 'candle15m', 'candle30m', 'candle1h', 'candle4h', 'candle1d']:
        sql = f'set search_path = {schema};'
        sql += f'''
            SELECT '{table}' as table, symbol, date, open, close, low, high
            FROM {table}
            WHERE date >= %(start_date)s
              AND date < %(end_date)s
            ORDER BY date ASC;
        '''
        candles_tmp = read_sql(sql, conn, params={'start_date': start_date, 'end_date': end_date})
        candles = pd.concat([candles, candles_tmp])

## Event

In [7]:
COUNTRY_1, COUNTRY_2 = SYMBOL.split('/')

In [9]:
with psycopg2.connect(dsn) as conn:
    sql = f'set search_path = {schema};'
    sql += '''
        SELECT date, country, name, actual_value, forecast_value, is_positive
        FROM event
        WHERE importance = 3
        AND country = ANY(%(countries)s::text[])
    '''
    event = read_sql(sql, conn, params={'countries': [COUNTRY_1, COUNTRY_2]})

In [11]:
months = ['(jan.)', '(févr.)', '(mar)', '(avr)', '(mai)', '(juin)', '(juill.)', '(août)', '(sept.)', '(oct.)', '(nov.)', '(déc)']
regex_month = re.compile(r'|'.join(months).replace('(', '\(').replace(')', '\)').replace('.', '\.'))

periods = [' (t1)', ' (t2)', ' (t3)', ' (t4)', ' m1', ' m2', ' m3', ' m4', ' m5', ' m6', ' m7', ' m8', ' m9', ' m10', ' m11', ' m12']
regex_period = re.compile(r'|'.join(periods).replace('(', '\(').replace(')', '\)'))

In [12]:
event['processed_name'] = event['name'].str.lower()
event['processed_name'] = event['processed_name'].str.replace(regex_month, '')
event['processed_name'] = event['processed_name'].str.replace(regex_period, '')
event['processed_name'] = event['processed_name'].str.strip()

In [13]:
event['processed_name'].value_counts()[:10]

stocks de pétrole brut                              571
inscriptions hebdomadaires au chômage               571
ipc (annuel)                                        263
discours de draghi, président de la bce             209
promesses de ventes de logements (mensuel)          132
rapport jolts - nouvelles offres d'emploi           132
indice pmi non manufacturier de l'ism               132
créations d'emplois dans le secteur non agricole    132
indice pmi manufacturier de l'ism                   132
taux de chômage                                     132
Name: processed_name, dtype: int64

# Show data
## Dates to hide

In [12]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

def compute_datetime_to_hide(start, end):
    start_dt = datetime.strptime(start, '%Y-%m-%d')
    end_dt = datetime.strptime(end, '%Y-%m-%d')
    
    fridays = list()
    for date in rrule(WEEKLY, byweekday=FR, dtstart=start_dt, until=end_dt):
        fridays.append(date)
    sundays = list()
    for date in rrule(WEEKLY, byweekday=SU, dtstart=start_dt, until=end_dt):
        sundays.append(date)
    
    date_ranges = list()
    if fridays[0] > sundays[0]:
        del sundays[0]
    if fridays[-1] > sundays[-1]:
        del fridays[-1]
    
    datetime_to_hide = list()
    for fri, sun in zip(fridays, sundays):
        res = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in datetime_range(fri.replace(hour=22),
                                                                         sun.replace(hour=19, minute=30),
                                                                         timedelta(minutes=5))]
        datetime_to_hide.extend(res)
    return datetime_to_hide

## Methods to plot result

In [15]:
def filter_data(candles, table, symbol=None, start_date=None, end_date=None):
    mask = (candles['table'] == table)
    if symbol:
        mask = mask & (candles['symbol'] == symbol)
    if start_date and end_date:
        mask = mask & (candles['date'] >= start_date) & (candles['date'] < end_date)
    candles_to_show = candles[mask]
    return candles_to_show


def show_candle(candles):
    layout = go.Layout(
        autosize=True,
        width=1400,
        height=800,
        xaxis=go.layout.XAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True),
        xaxis2=go.layout.XAxis(linecolor = 'black',
                               linewidth = 1,
                               mirror = True),
        yaxis=go.layout.YAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True,
                              domain=[0, 0.2]),
        yaxis2=go.layout.YAxis(linecolor = 'black',
                               linewidth = 1,
                               mirror = True,
                               domain=[0.3, 1]),

    )
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(layout, xaxis2_rangeslider_visible=False)
    fig.add_trace(go.Candlestick(x=candles['date'],
                                 open=candles['open'],
                                 high=candles['high'],
                                 low=candles['low'],
                                 close=candles['close']),
                   row=2, col=1)
    
    fig.update_yaxes(fixedrange=False)
    #fig.update_xaxes(rangebreaks=[dict(values=compute_datetime_to_hide(start_date, end_date))])

    return fig
    

## Show result

In [16]:
app = JupyterDash()

app.layout = html.Div([
    html.H1(f'Candles between {start_date} and {end_date}'),
    html.Div([
        html.Div([
            html.Label('Symbol:', className='label'),
            dcc.RadioItems(
                id='symbols_radio',
                options=[{'label': x, 'value': x} for x in SYMBOLS],
                value='EUR/USD'
            )], className='col'),
        html.Div([
            html.Label('Period:', className='label'),
            dcc.RadioItems(
                id='period_radio',
                labelStyle={'display': 'inline-block'},
                options=[{'label': '5min', 'value': 'candle'},
                         {'label': '15min', 'value': 'candle15m'},
                         {'label': '30min', 'value': 'candle30m'},
                         {'label': '1h', 'value': 'candle1h'},
                         {'label': '4h', 'value': 'candle4h'},
                         {'label': '1d', 'value': 'candle1d'}],
                value='candle1d'
            )], className='col'),
        html.Div([
            html.Label('Indicators:', className='label'),
            dcc.Checklist(
                id='indicators_checklist',
                options=[{'label': 'bollinger bands', 'value': 'bollinger_bands'},
                        {'label': 'macd', 'value': 'macd'}],
                value=[]
            )], className='col'),
    ], className='row'),
    dcc.Graph(id='candles_graph'),
])

@app.callback(
    Output('candles_graph', 'figure'),
    Input('symbols_radio', 'value'),
    Input('period_radio', 'value'),
    Input('indicators_checklist', 'value'))
def update_figure(selected_symbol, selected_period, selected_indicators):
    filtered_data = filter_data(candles, selected_period, selected_symbol)
    fig = show_candle(filtered_data)
    if 'bollinger_bands' in selected_indicators:
        bb = BollingerBands(filtered_data, 'close')
        bb.compute()
        fig = bb.plot(fig)
    if 'macd' in selected_indicators:
        m = Macd(filtered_data, 'close')
        m.compute()
        fig = m.plot(fig)
    return fig

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


# Test indicators

In [19]:
SYMBOL = 'EUR/USD'
candles_for_indicator = candles[candles['symbol'] == SYMBOL]
#candles.set_index('date', inplace=True)

In [26]:
ma, bb_up, bb_down = bollinger_bands(candles_for_indicator, 'close')

In [20]:
macd_, signal, hist = macd(candles_for_indicator, 'close')

In [21]:
hist

1           NaN
13          NaN
19          NaN
26          NaN
32          NaN
         ...   
90    -0.000371
98    -0.000097
106    0.000361
114    0.001055
122    0.001435
Name: close, Length: 6546, dtype: float64

In [29]:
bb_down

1           NaN
15          NaN
22          NaN
45          NaN
51          NaN
         ...   
170    1.178107
181    1.179303
184    1.180845
194    1.181929
206    1.182523
Name: close, Length: 9987, dtype: float64

In [11]:
type(signal)

pandas.core.series.Series

In [3]:
candles_for_indicator

NameError: name 'candles_for_indicator' is not defined

In [24]:
candles_for_indicator['max'] = candles_for_indicator[['open', 'close', 'low', 'high']].max(axis=1)